In [1]:
# Import Dependencies
import pandas as pd

In [2]:
# Import our input dataset
athletes_df = pd.read_csv('Resources/Olympic Project.csv')
athletes_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [3]:
athletes_df.describe()

,ID,Age,Height,Weight,Year
count,271116.000000,261642.000000,210945.000000,208241.000000,271116.000000
mean,68248.954396,25.556898,175.338970,70.702393,1978.378480
std,39022.286345,6.393561,10.518462,14.348020,29.877632
min,1.000000,10.000000,127.000000,25.000000,1896.000000
25%,34643.000000,21.000000,168.000000,60.000000,1960.000000
50%,68205.000000,24.000000,175.000000,70.000000,1988.000000
75%,102097.250000,28.000000,183.000000,79.000000,2002.000000
max,135571.000000,97.000000,226.000000,214.000000,2016.000000


In [4]:
list(athletes_df.columns)

['ID',
 'Name',
 'Sex',
 'Age',
 'Height',
 'Weight',
 'Team',
 'NOC',
 'Games',
 'Year',
 'Season',
 'City',
 'Sport',
 'Event',
 'Medal']

In [5]:
# Create new athlete dataframe
athlete_count_df = athletes_df[['ID', 'Name', 'NOC', 'Year', 'Season', 'Event']]
athlete_count_df

,ID,Name,NOC,Year,Season,Event
0,1,A Dijiang,CHN,1992,Summer,Basketball Men's Basketball
1,2,A Lamusi,CHN,2012,Summer,Judo Men's Extra-Lightweight
2,3,Gunnar Nielsen Aaby,DEN,1920,Summer,Football Men's Football
3,4,Edgar Lindenau Aabye,DEN,1900,Summer,Tug-Of-War Men's Tug-Of-War
4,5,Christine Jacoba Aaftink,NED,1988,Winter,Speed Skating Women's 500 metres
...,...,...,...,...,...,...
271111,135569,Andrzej ya,POL,1976,Winter,Luge Mixed (Men)'s Doubles
271112,135570,Piotr ya,POL,2014,Winter,"Ski Jumping Men's Large Hill, Individual"
271113,135570,Piotr ya,POL,2014,Winter,"Ski Jumping Men's Large Hill, Team"
271114,135571,Tomasz Ireneusz ya,POL,1998,Winter,Bobsleigh Men's Four


In [6]:
# Drop duplicate events in the same year to remove athletes on the same team
individual_athlete_count_df = athlete_count_df.drop_duplicates(subset=['NOC', 'Year', 'Event'])
individual_athlete_count_df

,ID,Name,NOC,Year,Season,Event
0,1,A Dijiang,CHN,1992,Summer,Basketball Men's Basketball
1,2,A Lamusi,CHN,2012,Summer,Judo Men's Extra-Lightweight
2,3,Gunnar Nielsen Aaby,DEN,1920,Summer,Football Men's Football
3,4,Edgar Lindenau Aabye,DEN,1900,Summer,Tug-Of-War Men's Tug-Of-War
4,5,Christine Jacoba Aaftink,NED,1988,Winter,Speed Skating Women's 500 metres
...,...,...,...,...,...,...
271095,135559,Pawe Jan Zygmunt,POL,1998,Winter,"Speed Skating Men's 5,000 metres"
271096,135559,Pawe Jan Zygmunt,POL,2002,Winter,"Speed Skating Men's 5,000 metres"
271097,135559,Pawe Jan Zygmunt,POL,2002,Winter,"Speed Skating Men's 10,000 metres"
271098,135559,Pawe Jan Zygmunt,POL,2006,Winter,"Speed Skating Men's 5,000 metres"


In [7]:
# Groupby Country Code and Year and create new dataframe for total athlete count across all countries and olympics
country_count = individual_athlete_count_df.groupby(['NOC', 'Year']).size()
country_count_df = pd.DataFrame(country_count, columns=['athlete_count'])
country_count_df.head()

athlete_count
NOC Year               
AFG 1936              4
    1948              2
    1956              1
    1960             13
    1964              8

In [8]:
# Summer olympic participation count across all years
summer_count = individual_athlete_count_df[individual_athlete_count_df['Season'] == 'Summer'].groupby(['NOC', 'Year']).size()
summer_count_df = pd.DataFrame(summer_count, columns=['athlete_count'])
summer_count_df.head()

athlete_count
NOC Year               
AFG 1936              4
    1948              2
    1956              1
    1960             13
    1964              8

In [9]:
# Winter olympic participation count across all years
winter_count = individual_athlete_count_df[individual_athlete_count_df['Season'] == 'Winter'].groupby(['NOC', 'Year']).size()
winter_count_df = pd.DataFrame(winter_count, columns=['athlete_count'])
winter_count_df.head()

athlete_count
NOC Year               
AHO 1988              2
    1992              1
ALB 2006              3
    2010              2
    2014              2